# About

The purpose of this notebook is to generate the shapes used in the coxeter paper figure.
Shapes are generated using fresnel.

In [ ]:
import itertools
import math

import fresnel
import numpy as np
import pandas as pd
import PIL

### Convex polyhedron
The example convex polyhedron is a [truncated cube from fresnel-examples](https://github.com/glotzerlab/fresnel-examples/blob/master/01-Primitives/02-Convex-polyhedron-geometry.ipynb).

In [ ]:
device = fresnel.Device()
scene = fresnel.Scene(device)

In [ ]:
# first get cube verts
pm = [-1, 1]
cube_verts = list(itertools.product(pm, repeat=3))
trunc_cube_verts = []
# truncate by removing corners and adding vertices to edges
for p1, p2 in itertools.combinations(cube_verts, 2):
    # don't add points along any diagonals
    match = (p1[0] == p2[0], p1[1] == p2[1], p1[2] == p2[2])
    if match.count(False) == 1:  # only 1 coordinate changes, not a diagonal
        p1, p2 = np.array(p1), np.array(p2)
        vec = p2 - p1
        trunc_cube_verts.append(p1 + vec / 3)
        trunc_cube_verts.append(p1 + 2 * vec / 3)

In [ ]:
c1 = fresnel.color.linear([0.70, 0.87, 0.54]) * 0.8
c2 = fresnel.color.linear([0.65, 0.81, 0.89]) * 0.8


colors = {8: c1, 3: c2}
poly_info = fresnel.util.convex_polyhedron_from_vertices(trunc_cube_verts)
for idx, fs in enumerate(poly_info["face_sides"]):
    poly_info["face_color"][idx] = colors[fs]
cube = fresnel.geometry.ConvexPolyhedron(scene, poly_info, N=1)
cube.outline_width = 0.02
cube.material = fresnel.material.Material(
    color=fresnel.color.linear([0.25, 0.5, 0.9]), roughness=0.8
)

In [ ]:
cube.position[:] = [[0, 0, 0]]
cube.orientation[:] = [[0.80777943, 0.41672122, 0.00255412, 0.41692838]]

In [ ]:
cube.color[:] = fresnel.color.linear(np.array([142, 203, 98]) / 255)
cube.material.primitive_color_mix = 1.0
cube.color_by_face = 0.3

In [ ]:
scene.camera = fresnel.camera.fit(scene, view="front")
image = PIL.Image.fromarray(fresnel.preview(scene)[:], mode="RGBA")
image.save("trunc_cube.png")

### Polygons
A random set of vertices is used to show examples of both nonconvex polygons and convex spheropolygons.

In [ ]:
device = fresnel.Device()
scene = fresnel.Scene(device)

In [ ]:
nonconvex_polygon = fresnel.geometry.Polygon(
    scene,
    N=1,
    vertices=np.array(
        [[0, 0], [0.4, -0.4], [1, 0.2], [0.8, 1], [-0.4, 1], [-0.8, 0.4], [-0.6, -0.2]]
    )
    * 1.6,
)
nonconvex_polygon.material.color = fresnel.color.linear(np.array([108, 183, 203]) / 255)
nonconvex_polygon.material.solid = 1

nonconvex_polygon.position[:] = [[3, -0.5]]
nonconvex_polygon.angle[:] = [0]
nonconvex_polygon.outline_width = 0.05

In [ ]:
scene.camera = fresnel.camera.fit(scene, view="front")
image = PIL.Image.fromarray(fresnel.preview(scene)[:], mode="RGBA")
image.save("polygon.png")

In [ ]:
device = fresnel.Device()
scene = fresnel.Scene(device)

In [ ]:
spheropolygon = fresnel.geometry.Polygon(
    scene,
    N=1,
    vertices=np.array(
        [[0.4, -0.4], [1, 0.2], [0.8, 1], [-0.4, 1], [-0.8, 0.4], [-0.6, -0.2]]
    )
    * 1.3,
    rounding_radius=0.5,
)
spheropolygon.material.color = fresnel.color.linear(np.array([218, 151, 143]) / 255)
spheropolygon.material.solid = 1

spheropolygon.position[:] = [[10, 0]]
spheropolygon.angle[:] = [np.pi / 2]
spheropolygon.outline_width = 0.05

In [ ]:
scene.camera = fresnel.camera.fit(scene, view="front")
image = PIL.Image.fromarray(fresnel.preview(scene)[:], mode="RGBA")
image.save("spheropolygon.png")

### General mesh

To demonstrate a generic mesh, we triangulate this thicc minimal bun from https://www.thingiverse.com/thing:151081. 

In [ ]:
import os

import fresnel
import numpy as np
import plato as pl
import plato.draw as draw
import plato.draw.fresnel

In [ ]:
def draw_bun():

    data = np.load("data/low_poly_stanford_bunny/data.npz")
    vertices = data["vertices"]
    indices = data["indices"]
    colors = data["colors"]
    prim = draw.Mesh(vertices=vertices, indices=indices, colors=colors, outline=4e-2)
    rotation = [-0.795798, 0.58683366, -0.12027311, -0.08869123]
    return draw.Scene(prim, rotation=rotation, zoom=2)


bun = draw_bun()
bun = bun.convert(pl.draw.fresnel)
bun.save("bunny.png")